#Speech Emotion Recognition with MLP Classifier



# Install following libraries

In [4]:
!pip install librosa soundfile numpy sklearn pyaudio

  Using cached PyAudio-0.2.11.tar.gz (37 kB)
  Running setup.py clean for pyaudio
Failed to build pyaudio
    Running setup.py install for pyaudio: started
    Running setup.py install for pyaudio: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'D:\Anaconda\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Dell\\AppData\\Local\\Temp\\pip-install-9xit0o08\\pyaudio_0f2ad265d5a04a05a1695d621ef61d0f\\setup.py'"'"'; __file__='"'"'C:\\Users\\Dell\\AppData\\Local\\Temp\\pip-install-9xit0o08\\pyaudio_0f2ad265d5a04a05a1695d621ef61d0f\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Dell\AppData\Local\Temp\pip-wheel-7bxrzaf9'
       cwd: C:\Users\Dell\AppData\Local\Temp\pip-install-9xit0o08\pyaudio_0f2ad265d5a04a05a1695d621ef61d0f\
  Complete output (17 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  copying src\pyaudio.py -> build\lib.win-amd64-3.8
  running build_ext
  building '_portaudio' extension
  creating build\temp.

In [5]:
!pip install soundfile

# Make the necessary imports

In [6]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Define a function extract_feature to extract the mfcc, chroma, and mel features from a sound file. This function takes 4 parameters- the file name and three Boolean parameters for the three features:

* mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
* chroma: Pertains to the 12 different pitch classes
* mel: Mel Spectrogram Frequency

In [7]:
def extract_feature(file_name, mfcc, chroma, mel):
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    if chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
    return result

Now, let’s define a dictionary to hold numbers and the emotions available in the RAVDESS & TESS dataset, and a list to hold all 8 emotions- neutral,calm,happy,sad,angry,fearful,disgust,surprised.

In [8]:
# Emotions in the RAVDESS & TESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
# Emotions to observe
observed_emotions=['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']

# Load the data and extract features for each sound file

In [9]:
#Load the data and extract features for each sound file
def load_data(test_size = 0.2):
  x, y = [], []
  for folder in glob.glob('C:/Users/Dell/Desktop/set/Actor_*'):
    print(folder)
    for file in glob.glob(folder + '/*.wav'):
      file_name = os.path.basename(file)
      emotion = emotions[file_name.split('-')[2]]
      if emotion not in observed_emotions:
        continue
      feature = extract_feature(file, mfcc = True, chroma = True, mel = True)
      x.append(feature)
      y.append(emotion)
  return train_test_split(np.array(x), y, test_size = test_size,train_size= 0.75,random_state = 9)

# Split the Dataset
Time to split the dataset into training and testing sets! Let’s keep the test set 25% of everything and use the load_data function for this.

In [10]:
# Split the dataset
import time
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

C:/Users/Dell/Desktop/set\Actor_01
C:/Users/Dell/Desktop/set\Actor_02
C:/Users/Dell/Desktop/set\Actor_03
C:/Users/Dell/Desktop/set\Actor_04
C:/Users/Dell/Desktop/set\Actor_05
C:/Users/Dell/Desktop/set\Actor_06
C:/Users/Dell/Desktop/set\Actor_07
C:/Users/Dell/Desktop/set\Actor_08
C:/Users/Dell/Desktop/set\Actor_09
C:/Users/Dell/Desktop/set\Actor_10
C:/Users/Dell/Desktop/set\Actor_11
C:/Users/Dell/Desktop/set\Actor_12
C:/Users/Dell/Desktop/set\Actor_13
C:/Users/Dell/Desktop/set\Actor_14
C:/Users/Dell/Desktop/set\Actor_15
C:/Users/Dell/Desktop/set\Actor_16
C:/Users/Dell/Desktop/set\Actor_17
C:/Users/Dell/Desktop/set\Actor_18
C:/Users/Dell/Desktop/set\Actor_19
C:/Users/Dell/Desktop/set\Actor_20
C:/Users/Dell/Desktop/set\Actor_21
C:/Users/Dell/Desktop/set\Actor_22
C:/Users/Dell/Desktop/set\Actor_23
C:/Users/Dell/Desktop/set\Actor_24


#Observe the shape of the training and testing datasets:

In [11]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(1749, 583)


# Number of features extracted.

In [12]:
# Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


# MLP Classifier

In [13]:
# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

#Fit/train the model.

In [14]:
# Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

# Predict the accuracy of our model

Let’s predict the values for the test set. This gives us y_pred (the predicted emotions for the features in the test set).

In [15]:
# Predict for the test set
y_pred=model.predict(x_test)

To calculate the accuracy of our model, we’ll call up the accuracy_score() function we imported from sklearn. Finally, we’ll round the accuracy to 2 decimal places and print it out.

In [16]:
# Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)
# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 72.73%


#classification Report

In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       angry       0.85      0.77      0.81        78
        calm       0.77      0.81      0.79        89
     disgust       0.65      0.62      0.64        45
     fearful       0.61      0.80      0.69        87
       happy       0.90      0.73      0.80        99
     neutral       0.76      0.73      0.75        56
         sad       0.74      0.52      0.61        89
   surprised       0.55      0.88      0.67        40

    accuracy                           0.73       583
   macro avg       0.73      0.73      0.72       583
weighted avg       0.75      0.73      0.73       583



# Confusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print (matrix)

[[60  1  3  7  0  1  1  5]
 [ 0 72  1  2  4  3  3  4]
 [ 1  1 28  8  0  1  1  5]
 [ 3  1  3 70  2  1  7  0]
 [ 5  4  3  5 72  1  3  6]
 [ 0  5  2  2  0 41  1  5]
 [ 2 10  2 19  1  5 46  4]
 [ 0  0  1  2  1  1  0 35]]


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
 
import seaborn
import matplotlib.pyplot as plt
 
 
def plot_confusion_matrix(data, labels, output_filename):
    """Plot confusion matrix using heatmap.
 
    Args:
        data (list of list): List of lists with confusion matrix data.
        labels (list): Labels which will be plotted across x and y axis.
        output_filename (str): Path to output file.
 
    """
    seaborn.set(color_codes=True)
    plt.figure(1, figsize=(9, 6))
 
    plt.title("Confusion Matrix")
 
    seaborn.set(font_scale=1.4)
    ax = seaborn.heatmap(data, annot=True, cmap="YlGnBu", cbar_kws={'label': 'Scale'})
 
    ax.set_xticklabels(labels)
    ax.set_yticklabels(labels)
 
    ax.set(ylabel="True Label", xlabel="Predicted Label")
 
    plt.savefig(output_filename, bbox_inches='tight', dpi=300)
    plt.close()
 
# define data
data =[[60,  1,  3,  7,  0,  1,  1,  5],
       [ 0, 72,  1,  2,  4,  3,  3,  4],
       [ 1,  1, 28,  8,  0,  1,  1,  5],
       [ 3,  1,  3, 70,  2,  1,  7,  0],
       [ 5,  4,  3,  5, 72,  1,  3,  6],
       [ 0,  5,  2,  2,  0, 41,  1,  5],
       [ 2, 10,  2, 19,  1,  5, 46,  4],
       [ 0,  0,  1,  2,  1,  1,  0, 35]]

# define labels
labels = ['neutral','calm','happy','sad','angry','fearful', 'disgust','surprised']
 
# create confusion matrix
plot_confusion_matrix(data, labels, "confusion_matrix.png")

In [29]:
# Writing different model files to file
with open( 'modelForPrediction5.sav', 'wb') as f:
    pickle.dump(model,f)

#Thank You